In [2]:
%load_ext autoreload
%autoreload 2

from utils import *
pdf_path = "mcs2024.pdf"

In [3]:
with open('json_files/scraping_base.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

# Convert serialized dictionaries back to DataFrames


def convert_dict_to_df(d):
    if isinstance(d, list) and all(isinstance(i, dict) for i in d):
        return pd.DataFrame(d)
    elif isinstance(d, list):
        return [convert_dict_to_df(i) for i in d]
    elif isinstance(d, dict):
        return {k: convert_dict_to_df(v) for k, v in d.items()}
    else:
        return d


scraping_base = convert_dict_to_df(loaded_scraping_base)

for material in list(scraping_base.keys()):
    scraping_base[material]['tables'] = []

material_test = ['Titanium']

In [4]:
import pandas as pd
import warnings

# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Suppress specific FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning,
                        message="The 'method' keyword in Series.replace is deprecated")
warnings.filterwarnings("ignore", category=FutureWarning,
                        message="Calling float on a single element Series is deprecated")

# Your existing code
for material in material_test:
    if isinstance(scraping_base[material]['tables'], pd.DataFrame):
        scraping_base[material]['tables'] = scraping_base[material]['tables'].to_dict(
            orient='records')
    else:
        scraping_base[material]['tables'] = []

    selected_ps, contents = scraping_base[material]['pages_num'], scraping_base[material]['pages_content']
    i = 0

    for selected_p in selected_ps:
        content = contents[i]
        i += 1
        result = extract_table(selected_p - 1, content, 2)
        if result is None or len(result[0]) == 0:
            continue

        list_of_table_df, list_of_bbox, page = result
        table_dfs = table_to_df(list_of_table_df, list_of_bbox, page, pdf_path)

        if not table_dfs:
            continue

        for table_df, bbox in zip(table_dfs, list_of_bbox):
            table = table_df.sort_values('bottom')
            # print(table)
            # print("*"*40)
            # print()
            scraping_base[material]['tables'].append((table.to_dict(orient='records'), bbox, page))

In [5]:
table

,text,bottom,text_1,text_2,text_3
31,None,103.66752,Mine production,Mine production,Reserves
32,None,114.70752,2022,2023,None
0,Ilmenite:,125.17650,None,None,None
1,United States2,136.15650,200,200,"2,000"
2,Australia,147.19650,400,400,"0180,000"
3,Brazil,158.17650,28,54,"43,000"
4,Canada1,169.15650,520,500,"52,000"
5,China,180.19650,"3,140","3,100","210,000"
6,India,191.17650,210,210,"85,000"
7,Kenya,202.15650,190,140,130


In [6]:
res_df = result[0][0]

In [1]:
# res_df[abs(res_df['bottom'] - 301.1565) < 0.1]

In [90]:
target_phrases = ['World total (ilmenite and rutile, rounded)', 'World total (rounded)', 'World total']

comb_temp_df  = pd.read_csv('sample_comb_df.csv')

# Find rows containing target phrases
for phrase in target_phrases:
    if phrase == 'World total (ilmenite and rutile, rounded)':
        print('WEIRD')
    target_rows = comb_temp_df[comb_temp_df['text'].str.contains(phrase, case=False, na=False, regex=False)]
    if not target_rows.empty:
        if phrase == 'World total (ilmenite and rutile, rounded)':
            print('WEIRD2')
            print(target_rows)
        # Update bbox_bottom_final to the bottom of the first detected phrase
        bbox_bottom_final = float(target_rows['bottom'].min())
        print(bbox_bottom_final)
        break  # Stop after finding the first match

WEIRD
WEIRD2
                                           text    x0     x1       top  \
122  World total (ilmenite and rutile, rounded)  54.0  232.0  445.1565   

           doctop    bottom  upright  height  width direction  margin_top  
122  150167.49006  455.1765     True   10.02  178.0       ltr        21.0  
455.1765


In [76]:
import pandas as pd

target_phrases = ['World total (ilmenite and rutile, rounded)', 'World total (rounded)', 'World total']

comb_temp_df = pd.read_csv('sample_comb_df.csv')

# # Print unique values in the 'text' column
# print("Unique values in 'text' column:")
# print(comb_temp_df['text'].unique())

# Find rows containing target phrases
for phrase in target_phrases:
    print(f"Searching for phrase: '{phrase}'")
    # Use regex for more flexible matching
    target_rows = comb_temp_df[comb_temp_df['text'].str.contains(phrase, case=False, na=False, regex=False)]
    if not target_rows.empty:
        print(f"Found match for: '{phrase}'")
        print(target_rows)
        bbox_bottom_final = float(target_rows['bottom'].min())
        break  # Stop after finding the first match
    else:
        print(f"No match found for: '{phrase}'")

# If no match was found
if target_rows.empty:
    print("No matching phrases found in the DataFrame.")

Searching for phrase: 'World total (ilmenite and rutile, rounded)'
Found match for: 'World total (ilmenite and rutile, rounded)'
                                           text    x0     x1       top  \
122  World total (ilmenite and rutile, rounded)  54.0  232.0  445.1565   

           doctop    bottom  upright  height  width direction  margin_top  
122  150167.49006  455.1765     True   10.02  178.0       ltr        21.0  


In [74]:
import pandas as pd

# Order phrases from most specific to least specific
target_phrases = [
    'World total (ilmenite and rutile, rounded)',
    'World total (ilmenite, rounded)',
    'World total (rutile, rounded)',
    'World total'
]

comb_temp_df = pd.read_csv('sample_comb_df.csv')

# Find rows containing target phrases
all_matches = []
for phrase in target_phrases:
    print(f"Searching for phrase: '{phrase}'")
    # Use regex for more flexible matching
    target_rows = comb_temp_df[comb_temp_df['text'].str.contains(phrase, case=False, na=False, regex=False)]
    if not target_rows.empty:
        print(f"Found match for: '{phrase}'")
        print(target_rows)
        all_matches.append((phrase, float(target_rows['bottom'].min())))
    else:
        print(f"No match found for: '{phrase}'")

# Print all matches
print("\nAll matches found:")
for match in all_matches:
    print(f"Phrase: '{match[0]}', Bottom: {match[1]}")

# If you still want to use only the first (most specific) match:
if all_matches:
    bbox_bottom_final = all_matches[0][1]
    print(f"\nUsing bottom value from first match: {bbox_bottom_final}")
else:
    print("No matching phrases found in the DataFrame.")

Searching for phrase: 'World total (ilmenite and rutile, rounded)'
Found match for: 'World total (ilmenite and rutile, rounded)'
                                           text    x0     x1       top  \
122  World total (ilmenite and rutile, rounded)  54.0  232.0  445.1565   

           doctop    bottom  upright  height  width direction  margin_top  
122  150167.49006  455.1765     True   10.02  178.0       ltr        21.0  
Searching for phrase: 'World total (ilmenite, rounded)'
Found match for: 'World total (ilmenite, rounded)'
                                text    x0     x1      top        doctop  \
69  World total (ilmenite, rounded)9  63.0  201.0  290.046  150012.37956   

      bottom  upright   height  width direction  margin_top  
69  301.1565     True  11.1105  138.0       ltr        21.0  
Searching for phrase: 'World total (rutile, rounded)'
Found match for: 'World total (rutile, rounded)'
                               text    x0     x1      top        doctop  \
118  Wor